## 190823
- find a sample case to run :
- select pdb_release_id from pdb_local where xml_path is not null and pdb_release_id in (select distinct(pdb_release_id) from pdb_chain where id in (select pdb_chain_id from raf where last_release_id = 18)) limit 5;  
+----------------+  
| pdb_release_id |    
+----------------+  
|          66518 |  
|          66755 |  
|          66756 |  
|          67708 |  
|         114278 |  
+----------------+  

- select * from pdb_chain where pdb_release_id = 66518 ;  
+--------+----------------+-------+----------------+  
| id     | pdb_release_id | chain | is_polypeptide |  
+--------+----------------+-------+----------------+  
| 398066 |          66518 | D     |              1 |  
| 398067 |          66518 | E     |              1 |  
| 398068 |          66518 | F     |              1 |  
| 398069 |          66518 | A     |              1 |  
| 398070 |          66518 | B     |              1 |  
| 398071 |          66518 | C     |              1 |  
+--------+----------------+-------+----------------+  

- select * from pdb_chain_dbref where pdb_chain_id in (select pdb_chain_id from raf where pdb_chain_id in (select id from pdb_chain where pdb_release_id = 66518) and raf_version_id = 3);  
+---------+--------------+---------+------------+--------------+----------------+--------------+-----------------+---------------+  
| id      | pdb_chain_id | db_name | db_code    | db_accession | db_align_start | db_align_end | pdb_align_start | pdb_align_end |  
| 5539951 |       398069 | UNP     | HIS6_THEMA |              |              1 |          253 |               0 |             252 |  
| 5539952 |       398071 | UNP     | HIS6_THEMA |              |              1 |          253 |               0 |             252 |  
| 5539953 |       398067 | UNP     | HIS6_THEMA |              |              1 |          253 |               0 |             252 |  
| 5539954 |       398070 | UNP     | HIS5_THEMA |              |              1 |          201 |               0 |             200 |  
| 5539955 |       398066 | UNP     | HIS5_THEMA |              |              1 |          201 |               0 |             200 |  
| 5539956 |       398068 | UNP     | HIS5_THEMA |              |              1 |          201 |               0 |             200 |  
+---------+--------------+---------+------------+--------------+----------------+--------------+-----------------+---  
- select * from uniprot where long_id = 'HIS6_THEMA' and is_obsolete = 0;  
| id       | long_id    | is_swissprot | seq_version | seq_date   | entry_name | is_obsolete |   
92046954 | HIS6_THEMA |            1 |           1 | 1999-11-01 | Imidazole glycerol phosphate synthase subunit HisF |           0 |   

- select * from uniprot_seq where uniprot_id = 92046954;  
+------------+--------+  
| uniprot_id | seq_id |  
+------------+--------+  
|   92046954 |  71862 |  
+------------+--------+  

- from astral_seq: 
"mlakriiacldvkdgrvvkgtnfenlrdsgdpvelgkfyseigidelvflditasvekrktmlelvekvaeqidipftvgggihdfetaselilrgadkvsintaavenpslitqiaqtfgsqavvvaidakrvdgefmvftysgkkntgillrdwvvevekrgageilltsidrdgtksgydtemirfvrplttlpiiasggagkmehfleaflagadaalaasvfhfreidvrelkeylkkhgvnvrlegl"  
- from raf: 1-253

- db_align is 1-indexed
- pdb_align is 0-indexed

## 190825
- Make a new table compare_seq with the following columns:
  - pdb_release_id
  - pdb_chain_id
  - raf_available
  - long_id
  - uniprot_id
  - seq_id
  - is_same

## 190829
`CREATE TABLE compare_seq (pdb_release_id INT NOT NULL,pdb_chain_id INT NOT NULL,raf_available Bool NOT NULL,long_id TEXT NOT NULL, uniprot_id INT NOT NULL,seq_id INT NOT NULL,is_same Bool NOT NULL);`

## 190830
- continue writing functionCompareUniprotSeqres

## 190903
- read the CAGI assessment paper
- complete the string comparison and update part of function CompareUniprotSeqres
- group meeting

## 190904
- CompareUniprotSeqres 66518

## 190905
- some issue arose while runnning the CompareUniprotSeqres on all the entries
- specifically, there're cases where db_align is 0-indexed instead of 1-indexed and lead to incorrect substring
  for now, write an exception to add 1 if db_align_start is zero.
- there're also cases where the pdb_align_start is NULL, which shouldn't be the case
- sanity check on whether I'm selecting the desired subset of the database

The code below check the number of raf entries with last_release_id of 18 and have a version=3 raf entry
`select count(*) from raf where pdb_chain_id in (select distinct(pdb_chain_id) from raf where last_release_id = 18) and first_release_id is null and last_release_id is null and raf_version_id = 3;`  
+----------+  
| count(*) |  
+----------+  
|   207690 |  
+----------+  

The code below check the total number of raf_version = 3 entries:
`select count(*) from raf where raf_version_id = 3 and last_release_id is null and first_release_id is null;`
+----------+  
| count(*) |  
+----------+  
|   641695 |  
+----------+    

The code below check whether there're pdb_chain_id with release_id of 18 but does not have a raf_version_id = 3 entry  
`select count(*) from raf where pdb_chain_id in (select distinct(pdb_chain_id) from raf where last_release_id = 18) and pdb_chain_id not in (select distinct(pdb_chain_id) from raf where pdb_chain_id in (select distinct(pdb_chain_id) from raf where last_release_id = 18) and first_release_id is null and last_release_id is null and raf_version_id = 3);
+----------+
| count(*) |
+----------+
|        0 |
+----------+
`
 
and vice versa  
`select count(*) from raf where pdb_chain_id in (select distinct(pdb_chain_id) from raf where pdb_chain_id in (select distinct(pdb_chain_id) from raf where last_release_id = 18) and first_release_id is null and last_release_id is null and raf_version_id = 3) and pdb_chain_id not in (select distinct(pdb_chain_id) from raf where last_release_id = 18);
+----------+
| count(*) |
+----------+
|        0 |
+----------+
1 row in set (35.68 sec)
`

and thus all the raf with release_id = 18 has a corresponding raf_version = 3 entry


Then, check on how many pdb_align_start/end have null in pdb_chain_dbref with release id of 18 . 
`select count(*) from pdb_chain_dbref where pdb_align_start IS NULL and pdb_chain_id in (select distinct(pdb_chain_id) from raf where last_release_id = 18);
+----------+
| count(*) |
+----------+
|      274 |
+----------+
1 row in set (13.34 sec)
`
`
MySQL [scop]> select count(*) from pdb_chain_dbref where pdb_align_end IS NULL and pdb_chain_id in (select distinct(pdb_chain_id) from raf where last_release_id = 18);
+----------+
| count(*) |
+----------+
|      217 |
+----------+
1 row in set (14.38 sec)
`

`
MySQL [scop]> select count(*) from pdb_chain_dbref where pdb_align_end IS NULL and pdb_align_start IS NULL and pdb_chain_id in (select distinct(pdb_chain_id) from raf where last_release_id = 18);
+----------+
| count(*) |
+----------+
|      170 |
+----------+
`  

`
select count(*) from pdb_chain_dbref where pdb_chain_id in (select distinct(pdb_chain_id) from raf where last_release_id = 18);
+----------+
| count(*) |
+----------+
|   216161 |
+----------+
1 row in set (25.02 sec)
`

And so out of the 216161 entries within pdb_release_id = 18, there're ~300 entries with pdb_alignment_issue, for now, write an exception to exclude these entries from execution.

To see how many pdb_release_id is needed to be iterated from:
`select distinct(pdb_release_id) from pdb_chain where id in (select distinct(pdb_chain_id) from raf where last_release_id = 18);
88499 rows in set (25.70 sec)`

refine the pdb_release_id selection, the previous error results from pdb_chain_id where the pdb_release_id = 11